# NMS (Non Maximum Suppression)

객체 인식(Object Detection)에서 **NMS(Non-Maximum Suppression, 비최대 억제)**는 모델이 예측한 다수의 중복된 바운딩 박스 중에서 가장 신뢰도 높은 박스를 선택하고 나머지를 제거하여 최종 결과를 정제하는 후처리 기법입니다.

## NMS의 목적

1. 중복 제거: 하나의 객체에 대해 여러 개의 바운딩 박스가 예측되는 경우, 가장 적절한 하나의 박스만을 남기고 나머지를 제거합니다.

2. 정확도 향상: 불필요한 중복 박스를 제거함으로써 검출 결과의 정확도를 높이고, false positive를 줄입니다.

3. 계산 효율성 증대: 최종 결과에 포함될 박스 수를 줄여 후속 처리나 시각화 단계에서의 계산 부담을 감소시킵니다.

##  NMS의 절차

1. 바운딩 박스 및 신뢰도 점수 획득: 모델이 예측한 모든 바운딩 박스와 해당 박스의 신뢰도(confidence) 점수를 수집합니다.

2. 신뢰도 기준 정렬: 모든 박스를 신뢰도 점수를 기준으로 내림차순 정렬합니다.

3. 최고 신뢰도 박스 선택: 가장 높은 신뢰도 점수를 가진 박스를 선택합니다.

4. IOU(Intersection over Union) 계산: 선택된 박스와 나머지 박스들 간의 IOU를 계산합니다. IOU는 두 박스의 겹치는 영역의 비율을 나타냅니다.

5. IOU 임계값 적용 및 박스 제거: IOU가 미리 정의된 임계값(예: 0.5) 이상인 박스들은 중복으로 간주하여 제거합니다.

6. 반복 수행: 남은 박스들에 대해 위 과정을 반복하여 최종적으로 중복이 제거된 박스들만을 남깁니다.



### IOU(Intersection over Union)란?

IOU는 두 바운딩 박스 간의 겹치는 정도를 측정하는 지표로, 다음과 같이 계산됩니다:


$$\mathrm{IoU}(A, B) = \frac{|A \cap B|}{|A \cup B|}$$


IOU 값은 0에서 1 사이의 값을 가지며, 1에 가까울수록 두 박스가 거의 동일한 위치를 나타냅니다.

## NMS의 변형 기법

- Soft-NMS: 기존 NMS는 IOU 임계값을 초과하는 박스를 완전히 제거하는 반면, Soft-NMS는 이러한 박스들의 신뢰도 점수를 감소시켜 보다 부드러운 억제를 수행합니다. 이를 통해 일부 유용한 박스가 완전히 제거되는 것을 방지할 수 있습니다.

- Class-wise NMS: 객체의 클래스별로 NMS를 적용하여 서로 다른 클래스 간의 박스 억제를 방지합니다.

## Python NMS 구현

In [ ]:
def compute_iou(box1, box2):
    # box: [x1, y1, x2, y2]
    x1, y1, x2, y2 = box1
    x1_p, y1_p, x2_p, y2_p = box2

    # 1) 교집합 좌표 계산
    inter_x1 = max(x1, x1_p)
    inter_y1 = max(y1, y1_p)
    inter_x2 = min(x2, x2_p)
    inter_y2 = min(y2, y2_p)

    # 2) 교집합 너비·높이 계산
    inter_width = inter_x2 - inter_x1
    inter_height = inter_y2 - inter_y1
    if inter_width <= 0 or inter_height <= 0:
        return 0.0
    inter_area = inter_width * inter_height

    # 3) 박스 면적 계산
    area1 = (x2 - x1) * (y2 - y1)
    area2 = (x2_p - x1_p) * (y2_p - y1_p)

    # 4) 합집합 면적 = 면적1 + 면적2 - 교집합
    union_area = area1 + area2 - inter_area

    # 5) IoU 반환
    return inter_area / union_area


def nms(boxes, scores, iou_threshold):
    """
    boxes: list of [x1, y1, x2, y2]
    scores: list of floats
    iou_threshold: float
    returns: list of kept indices
    """
    # 1) 인덱스 리스트 생성
    indices = list(range(len(scores)))

    # 2) 점수 기준 내림차순 정렬 (버블소트 형태)
    for i in range(len(indices)):
        for j in range(i+1, len(indices)):
            if scores[indices[j]] > scores[indices[i]]:
                indices[i], indices[j] = indices[j], indices[i]

    keep_indices = []

    # 3) 남은 인덱스가 없을 때까지 반복
    while indices:
        current = indices[0]
        keep_indices.append(current)

        new_indices = []
        # 4) 나머지 박스들과 IoU 비교
        for idx in indices[1:]:
            iou = compute_iou(boxes[current], boxes[idx])
            if iou < iou_threshold:
                new_indices.append(idx)

        # 5) 인덱스 업데이트
        indices = new_indices

    return keep_indices

## 실습

이전 실습의 yolov11n_voc나 yolov12n_voc의 onnx 추론 코드에서 NMS 함수를 위 함수로 변환하고, score threshold는 별도의 함수로 분리하여 추론하는 코드를 작성해보시오.